In [ ]:
from __future__ import division
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import scipy as scp
import pylab as pyl

import warnings
warnings.filterwarnings('ignore')
np.random.seed(1234)

%matplotlib inline
%load_ext autoreload
%autoreload 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
if not os.path.isdir('SinkhornvsDampedNewton_images'):
    os.makedirs('SinkhornvsDampedNewton_images')

In [ ]:
"""To compute distance matrix"""
def distmat(x,y):
    return np.sum( x**2,0 )[:,None] + np.sum( y**2,0 )[None,:] - 2*x.transpose().dot(y)

"""To Normalise a vector"""
normalize = lambda a: a/np.sum( a )

"""To Compute P"""
def GetP(u,K,v):
    return u[:,None]*K*v[None,:]

def plotp(x, col,plt, scale=200, edgecolors="k"):
  return plt.scatter( x[0,:], x[1,:], s=scale, edgecolors=edgecolors,  c=col, cmap='plasma', linewidths=2 )

In [ ]:
import computational_OT

# Sinkhorn iteration function

In [ ]:
def sinkhorn(epsilons,N,x,y,iterations = 1000):
    # Sinkhorn
    print("Sinkhorn.... ")
    SinkhornP                  = []
    results_Sinkhorn           = []
    times_Sinkhorn             = []
    Pmatrix_dist_linVSsinkhorn = []
    for eps in epsilons:

        print( "Sinkhorn for epsilon = "+str(eps)+":" )    
        #Cost matrix
        C = distmat(x,y)
        
        # a and b
        a = normalize(np.ones(N[0]))
        a = a.reshape(a.shape[0],-1)
        b = normalize(np.ones(N[1]))
        b = b.reshape(b.shape[0],-1)



        #Kernel
        K = np.exp(-C/eps)


        print("Doing for (",N[0],N[1],").")
        print( " |- Iterating")

        #Inflating
        u = a
        v = b

        start     =time.time()
        Optimizer = computational_OT.Sinkhorn(K,a,b,u,v,eps)
        out       = Optimizer._update(maxiter = iterations)
        results_Sinkhorn.append(out)
        end = time.time()
        times_Sinkhorn.append(1e-3*(end-start))
        print( " |- Computing P")
        print( "" )
        SinkhornP.append(GetP(np.exp(out['potential_f']/eps),K,np.exp(out['potential_g']/eps)))
    return {
        'results_list': results_Sinkhorn,
        'time_stamps' : times_Sinkhorn,
        'Ps'          : SinkhornP
    }
    

# Damped Newton(without precondition) iteration function

In [ ]:
def dampednewton(epsilons,N,x,y,rho = 0.95,c = 0.05, iterations = 50 ):
    DampedNewtonP=[]
    results_DampedNewton  = []
    times_DampedNewton    = []
    Hessians_DampedNewton = []


    for eps in epsilons:
        print("Damped Newton for epsilon = "+str(eps)+":")    
        #Cost matrix
        C = distmat(x,y)

        # a and b
        a = normalize(np.ones(N[0]))
        a=a.reshape(a.shape[0],-1)
        b = normalize(np.ones(N[1]))
        b=b.reshape(b.shape[0],-1)  

        #Kernel
        K=np.exp(-C/eps)
        f,g=a,b

        print("Doing for (",N[0],N[1],").")
        print( " |- Iterating")  
        start=time.time()
        Optimizer=computational_OT.DampedNewton(K,a,b,f,g,eps,rho,c)
        out=Optimizer._update(maxiter = iterations)
        results_DampedNewton.append(out)
        end=time.time()
        times_DampedNewton.append(1e-3*(end-start))
        print( " |- Computing P")
        
        DampedNewtonP.append(GetP(np.exp(out['potential_f']/eps),K,np.exp(out['potential_g']/eps)))
        print( " |- Recording (unstabilized) Hessian \n")

        mat  = -eps*Optimizer.Hessian
        diag = 1/np.sqrt( np.vstack( (a,b) ) ).flatten()
        mat = diag[:,None]*mat*diag[None,:]
        Hessians_DampedNewton.append( mat )

    return {
        'results_list': results_DampedNewton,
        'time_stamps' : times_DampedNewton,
        'Ps'          : DampedNewtonP,
        'Hessians'    : Hessians_DampedNewton 
    }
    

# Damped Newton with preconditioning iteration function

In [ ]:
def dampednewtonprecondition(epsilons,N,x,y, null_vector, precond_vectors, rho = 0.95, c = 0.05,  iterations = 500, iter_inv = 30, optimizer = 'cg', ver = None, debug = False):
    reset_starting_point = True
    final_modified_Hessians = []
    DampedNewtonwithprecondP = []
    results_DampedNewtonwithprecond  = []
    times_DampedNewtonwithprecond    = []


    f, g = None, None
    for eps in epsilons:
        print( "Damped Newton with preconditioning for epsilon = "+str(eps)+":" )    
        # Cost matrix
        C = distmat(x,y)

        # a and b
        a = normalize( np.ones(N[0]) )
        a = a.reshape( a.shape[0],-1 )
        b = normalize( np.ones(N[1]) )
        b = b.reshape( b.shape[0],-1 )

        #Kernel
        K = np.exp(-C/eps)

        if (f is None) or (g is None): 
            f,g = a,b

        print( "Doing for (",N[0],N[1],")." )
        
        print( " |- Iterating" )  
        start = time.time()
        Optimizer = computational_OT.DampedNewton_With_Preconditioner( K,a,b,f,g,eps,rho,c,null_vector,precond_vectors[:] )
        out = Optimizer._update( maxiter = iterations, iterative_inversion = iter_inv, version = ver, debug = debug, optType = optimizer )
        results_DampedNewtonwithprecond.append( out )
        end = time.time()
        
        times_DampedNewtonwithprecond.append(1e-3*(end-start))
        print( " |- Computing P" )

        if not reset_starting_point:
            f = Optimizer.x[:a.shape[0]]
            
            g = Optimizer.x[a.shape[0]:]
            # f = f.reshape( f.shape[0], -1)
            # g = g.reshape( g.shape[0], -1)
        
        
        
        DampedNewtonwithprecondP.append( GetP(np.exp(out['potential_f']/eps),K,np.exp(out['potential_g']/eps)) )
        final_modified_Hessians.append( Optimizer.modified_Hessian )
    return {
        'results_list': results_DampedNewtonwithprecond,
        'time_stamps' : times_DampedNewtonwithprecond,
        'Ps'          : DampedNewtonwithprecondP,
        'Hessians'    : final_modified_Hessians 
    }
    

# 1. Comparison for Data size 400 and 500

In [ ]:
N = [ 400, 500 ]

In [ ]:
x     = np.random.rand(2,N[0])-0.5
theta = 2*np.pi*np.random.rand(1,N[1])
r     = 0.8+.2*np.random.rand(1,N[1])
y     = np.vstack((r*np.cos(theta),r*np.sin(theta)))

## I. Sinkhorn

In [ ]:
epsilons = [  0.03, 0.001, 0.0009, 0.00084 ]
results_sinkhorn = sinkhorn(epsilons,N,x,y, iterations = 20000)

### Error plot

In [ ]:
plt.figure(figsize = (20,7))

plt.subplot(2,1,1),
plt.title("$||P1 -a||_1+||P1 -b||_1$")
for i in range(len(results_sinkhorn['results_list'])):
  error = np.asarray(results_sinkhorn['results_list'][i]['error_a'])+np.asarray(results_sinkhorn['results_list'][i]['error_b'])
  plt.plot( error,label='Sinkhorn for $\epsilon=$'+ str(epsilons[i]), linewidth = 2)
plt.yscale( 'log')
plt.legend()
plt.savefig("SinkhornvsDampedNewton_images/ConvergenceSinkhornvaryingepsilon.png")
plt.show()

## II. Damped Newton without Preconditioning

In [ ]:
epsilons = [  0.5, 0.03 ]
results_DampedNewton = dampednewton(epsilons, N,x,y)

### Error plot

In [ ]:
plt.figure(figsize = (20,7))
plt.title("$$")
plt.title("$||P1 -a||_1+||P^T 1 -b||_1$")

for i in range(len(results_DampedNewton['results_list'])):
  error=np.asarray(results_DampedNewton['results_list'][i]['error_a'])+np.asarray(results_DampedNewton['results_list'][i]['error_b'])
  plt.plot( error,label='Damped Newton for $\epsilon=$'+ str(epsilons[i]), linewidth = 2)

plt.xlabel("Number of iterations")
plt.ylabel("Error in log-scale")
plt.legend()
plt.yscale( 'log')
plt.savefig("SinkhornvsDampedNewton_images/ErrorDampedNewtonwithoutPrecond.png")
plt.show()

print("\n Error plots can increase! The error is not the objective function!")

### Objective function

In [ ]:
plt.figure(figsize = (20,7))
plt.title("$$")

plt.title("Objective Function")

for i in range(len(results_DampedNewton['results_list'])):
  plt.plot( np.asarray(results_DampedNewton['results_list'][i]['objectives']),label='Damped Newton for $\epsilon=$'+ str(epsilons[i]), linewidth = 2)

plt.xlabel("Number of iterations")
plt.ylabel("Objective value")
plt.legend()
plt.savefig("SinkhornvsDampedNewton_images/ObjectiveDampedNewtonwithoutPrecond.png")
plt.show()


### Alpha plot


In [ ]:
plt.figure(figsize = (20,7))
plt.subplot(2,1,1),
plt.title("Alpha")

for i in range(len(results_DampedNewton['results_list'])):
  plt.plot( np.asarray(results_DampedNewton['results_list'][i]['linesearch_steps']),label='Damped Newton for $\epsilon=$'+ str(epsilons[i]), linewidth = 2)

plt.xlabel("Number of iterations")
plt.ylabel("Alpha in log-scale")
plt.legend()
# plt.yscale( 'log')
plt.savefig("SinkhornvsDampedNewton_images/AlphaDampedNewtonwithoutPrecond.png")
plt.show()



### Plotting spectrum as a function of $\varepsilon$

In [ ]:
def spectral_decomposition(mat):
    eig, v = np.linalg.eigh( mat )
    sorting_indices = np.argsort(eig)
    eig = eig[sorting_indices]
    v   = v[:, sorting_indices]
    
    print( "List of smallest eigenvalues: ", eig[:10])
    print( "List of largest  eigenvalues: ", eig[-10:])

    return eig,v

In [ ]:
eigs=[]
eigvecs=[]
for i in range(len(epsilons)):
    eps = epsilons[i]
    print("Spectral statistics of Hessian for epsilon="+str(eps))
    ev=spectral_decomposition( results_DampedNewton['Hessians'][i] )
    eigs.append(ev[0])
    eigvecs.append(ev[1])
    print("")


In [ ]:
fig,ax=plt.subplots(figsize=(20,3),nrows=1, ncols=len(epsilons), sharey=True)
plt.title("Histogram of eigenvalues.")
for i in range(len(epsilons)):
    ax[i].hist( eigs[i], 50)
    ax[i].set_title( " $\epsilon$: "+str(epsilons[i]))
    ax[i].set_xlabel("Eigenvalues")
    ax[i].set_yscale( "log" )
plt.subplots_adjust(wspace=0,hspace=0)
plt.savefig("SinkhornvsDampedNewton_images/eigenhistunstabilizedDampedNewtonwithoutPrecond.png")
plt.show()

### Actual preconditioning

In [ ]:
def build_preconditioners( num_eigs,modified_Hessian, ansatz=True ):
    # Diagonalize
    eigenvalues, eigenvectors = np.linalg.eigh( modified_Hessian )
    sorting_indices = np.argsort( eigenvalues )
    eigenvalues  = eigenvalues[sorting_indices]
    eigenvectors = eigenvectors[:, sorting_indices]
    # Form null vector
    if not ansatz:
        null_vector = eigenvectors[:, 0]
    else:
        null_vector = np.hstack( (np.ones(N[0]), -np.ones(N[1])) )
        norm = np.sqrt( N[0] + N[1] )
        null_vector = null_vector/norm
    # Form other vectors (only 13)
    n,m = eigenvectors.shape
    indices=[]
    for i in range(num_eigs//2):
        indices.append(m-i-1)
        indices.append(i+1)
    if num_eigs//2!=0:
        indices.append(m-1-num_eigs//2)
   
    precond_vectors = eigenvectors[:, indices ]
    precond_vectors = []
    for index in indices:
        precond_vectors.append( eigenvectors[:,index] )
    #
    return null_vector, precond_vectors

In [ ]:
num_eigs = 13
null_vector, precond_vectors = build_preconditioners( num_eigs, results_DampedNewton['Hessians'][-1], ansatz=False )

## III. Damped Newton with Preconditioning

### Iterative inversion = 2

#### rho = 0.5

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_DampedNewtonwithprecond = dampednewtonprecondition(epsilons, N, x, y, null_vector, precond_vectors, iterations = 500,iter_inv = 2, rho = 0.5 )

In [ ]:
plt.figure(figsize = (20,7))
plt.title("$$")
plt.title("$||P1 -a||_1+||P^T 1 -b||_1$")

for i in range(len(results_DampedNewtonwithprecond['results_list'])):
  error=np.asarray(results_DampedNewtonwithprecond['results_list'][i]['error_a'])+np.asarray(results_DampedNewtonwithprecond['results_list'][i]['error_b'])
  plt.plot( error,label='Damped Newton with preconditioning for $\epsilon=$'+ str(epsilons[i]), linewidth = 2)
  
plt.xlabel("Number of iterations")
plt.ylabel("Error in log-scale")
plt.legend()  
plt.yscale( 'log')
plt.savefig("SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_5itv2.png")
plt.show()

print("\n Error plots can increase! The error is not the objective function!")

#### rho = 0.7

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_DampedNewtonwithprecond = dampednewtonprecondition(epsilons, N, x, y, null_vector, precond_vectors, iterations = 500,iter_inv = 2, rho = 0.7 )

In [ ]:
plt.figure(figsize = (20,7))
plt.title("$$")
plt.title("$||P1 -a||_1+||P^T 1 -b||_1$")

for i in range(len(results_DampedNewtonwithprecond['results_list'])):
  error=np.asarray(results_DampedNewtonwithprecond['results_list'][i]['error_a'])+np.asarray(results_DampedNewtonwithprecond['results_list'][i]['error_b'])
  plt.plot( error,label='Damped Newton with preconditioning for $\epsilon=$'+ str(epsilons[i]), linewidth = 2)
  
plt.xlabel("Number of iterations")
plt.ylabel("Error in log-scale")
plt.legend()  
plt.yscale( 'log')
plt.savefig("SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_7itv2.png")
plt.show()

print("\n Error plots can increase! The error is not the objective function!")

#### rho = 0.9

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_DampedNewtonwithprecond = dampednewtonprecondition(epsilons, N, x, y, null_vector, precond_vectors, iterations = 500,iter_inv = 2, rho = 0.9 )

In [ ]:
plt.figure(figsize = (20,7))
plt.title("$$")
plt.title("$||P1 -a||_1+||P^T 1 -b||_1$")

for i in range(len(results_DampedNewtonwithprecond['results_list'])):
  error=np.asarray(results_DampedNewtonwithprecond['results_list'][i]['error_a'])+np.asarray(results_DampedNewtonwithprecond['results_list'][i]['error_b'])
  plt.plot( error,label='Damped Newton with preconditioning for $\epsilon=$'+ str(epsilons[i]), linewidth = 2)
  
plt.xlabel("Number of iterations")
plt.ylabel("Error in log-scale")
plt.legend()  
plt.yscale( 'log')
plt.savefig("SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_9itv2.png")
plt.show()

print("\n Error plots can increase! The error is not the objective function!")

### Iterative Inversion = 5

#### rho = 0.5

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_DampedNewtonwithprecond = dampednewtonprecondition(epsilons, N, x, y, null_vector, precond_vectors, iterations = 500,iter_inv = 5, rho = 0.5 )

In [ ]:
plt.figure(figsize = (20,7))
plt.title("$$")
plt.title("$||P1 -a||_1+||P^T 1 -b||_1$")

for i in range(len(results_DampedNewtonwithprecond['results_list'])):
  error=np.asarray(results_DampedNewtonwithprecond['results_list'][i]['error_a'])+np.asarray(results_DampedNewtonwithprecond['results_list'][i]['error_b'])
  plt.plot( error,label='Damped Newton with preconditioning for $\epsilon=$'+ str(epsilons[i]), linewidth = 2)
  
plt.xlabel("Number of iterations")
plt.ylabel("Error in log-scale")
plt.legend()  
plt.yscale( 'log')
plt.savefig("SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_5itv5.png")
plt.show()

print("\n Error plots can increase! The error is not the objective function!")

#### rho = 0.7

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_DampedNewtonwithprecond = dampednewtonprecondition(epsilons, N, x, y, null_vector, precond_vectors, iterations = 500,iter_inv = 5, rho = 0.7 )

In [ ]:
plt.figure(figsize = (20,7))
plt.title("$$")
plt.title("$||P1 -a||_1+||P^T 1 -b||_1$")

for i in range(len(results_DampedNewtonwithprecond['results_list'])):
  error=np.asarray(results_DampedNewtonwithprecond['results_list'][i]['error_a'])+np.asarray(results_DampedNewtonwithprecond['results_list'][i]['error_b'])
  plt.plot( error,label='Damped Newton with preconditioning for $\epsilon=$'+ str(epsilons[i]), linewidth = 2)
  
plt.xlabel("Number of iterations")
plt.ylabel("Error in log-scale")
plt.legend()  
plt.yscale( 'log')
plt.savefig("SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_7itv5.png")
plt.show()

print("\n Error plots can increase! The error is not the objective function!")

#### rho = 0.9

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_DampedNewtonwithprecond = dampednewtonprecondition(epsilons, N, x, y, null_vector, precond_vectors, iterations = 500,iter_inv = 5, rho = 0.9 )

In [ ]:
plt.figure(figsize = (20,7))
plt.title("$$")
plt.title("$||P1 -a||_1+||P^T 1 -b||_1$")

for i in range(len(results_DampedNewtonwithprecond['results_list'])):
  error=np.asarray(results_DampedNewtonwithprecond['results_list'][i]['error_a'])+np.asarray(results_DampedNewtonwithprecond['results_list'][i]['error_b'])
  plt.plot( error,label='Damped Newton with preconditioning for $\epsilon=$'+ str(epsilons[i]), linewidth = 2)
  
plt.xlabel("Number of iterations")
plt.ylabel("Error in log-scale")
plt.legend()  
plt.yscale( 'log')
plt.savefig("SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_9itv5.png")
plt.show()

print("\n Error plots can increase! The error is not the objective function!")

### Iterative Inversion = 10

#### rho = 0.5

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_DampedNewtonwithprecond = dampednewtonprecondition(epsilons, N, x, y, null_vector, precond_vectors, iterations = 500,iter_inv = 10, rho = 0.5 )

In [ ]:
plt.figure(figsize = (20,7))
plt.title("$$")
plt.title("$||P1 -a||_1+||P^T 1 -b||_1$")

for i in range(len(results_DampedNewtonwithprecond['results_list'])):
  error=np.asarray(results_DampedNewtonwithprecond['results_list'][i]['error_a'])+np.asarray(results_DampedNewtonwithprecond['results_list'][i]['error_b'])
  plt.plot( error,label='Damped Newton with preconditioning for $\epsilon=$'+ str(epsilons[i]), linewidth = 2)
  
plt.xlabel("Number of iterations")
plt.ylabel("Error in log-scale")
plt.legend()  
plt.yscale( 'log')
plt.savefig("SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_5itv10.png")
plt.show()

print("\n Error plots can increase! The error is not the objective function!")

#### rho = 0.7

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_DampedNewtonwithprecond = dampednewtonprecondition(epsilons, N, x, y, null_vector, precond_vectors, iterations = 500,iter_inv = 10, rho = 0.7 )

In [ ]:
plt.figure(figsize = (20,7))
plt.title("$$")
plt.title("$||P1 -a||_1+||P^T 1 -b||_1$")

for i in range(len(results_DampedNewtonwithprecond['results_list'])):
  error=np.asarray(results_DampedNewtonwithprecond['results_list'][i]['error_a'])+np.asarray(results_DampedNewtonwithprecond['results_list'][i]['error_b'])
  plt.plot( error,label='Damped Newton with preconditioning for $\epsilon=$'+ str(epsilons[i]), linewidth = 2)
  
plt.xlabel("Number of iterations")
plt.ylabel("Error in log-scale")
plt.legend()  
plt.yscale( 'log')
plt.savefig("SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_7itv10.png")
plt.show()

print("\n Error plots can increase! The error is not the objective function!")

#### rho = 0.9

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_DampedNewtonwithprecond = dampednewtonprecondition(epsilons, N, x, y, null_vector, precond_vectors, iterations = 500,iter_inv = 10, rho = 0.9 )

In [ ]:
plt.figure(figsize = (20,7))
plt.title("$$")
plt.title("$||P1 -a||_1+||P^T 1 -b||_1$")

for i in range(len(results_DampedNewtonwithprecond['results_list'])):
  error=np.asarray(results_DampedNewtonwithprecond['results_list'][i]['error_a'])+np.asarray(results_DampedNewtonwithprecond['results_list'][i]['error_b'])
  plt.plot( error,label='Damped Newton with preconditioning for $\epsilon=$'+ str(epsilons[i]), linewidth = 2)
  
plt.xlabel("Number of iterations")
plt.ylabel("Error in log-scale")
plt.legend()  
plt.yscale( 'log')
plt.savefig("SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_9itv10.png")
plt.show()

print("\n Error plots can increase! The error is not the objective function!")

#### rho = 0.95

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_DampedNewtonwithprecond = dampednewtonprecondition(epsilons, N, x, y, null_vector, precond_vectors, iterations = 500,iter_inv = 10, rho = 0.95 )

In [ ]:
plt.figure(figsize = (20,7))
plt.title("$$")
plt.title("$||P1 -a||_1+||P^T 1 -b||_1$")

for i in range(len(results_DampedNewtonwithprecond['results_list'])):
  error=np.asarray(results_DampedNewtonwithprecond['results_list'][i]['error_a'])+np.asarray(results_DampedNewtonwithprecond['results_list'][i]['error_b'])
  plt.plot( error,label='Damped Newton with preconditioning for $\epsilon=$'+ str(epsilons[i]), linewidth = 2)
  
plt.xlabel("Number of iterations")
plt.ylabel("Error in log-scale")
plt.legend()  
plt.yscale( 'log')
plt.savefig("SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_95itv10.png")
plt.show()

print("\n Error plots can increase! The error is not the objective function!")